## set up AWA environment and vocs

In [1]:
from plugins.environments.awa_environment import AWAEnvironment
from plugins.interfaces.awa_interface import AWAInterface

# import data from csv file
import pandas as pd
variable_file = "plugins/environments/awa_variables.csv"
observable_file = "plugins/environments/awa_observables.csv"

env = AWAEnvironment(variable_file, observable_file, interface=AWAInterface(), target_charge=1.0)

In [2]:
env.variables

{'AWA:Drive:DS1:Ctrl': [500.0, 550.0],
 'AWA:Drive:DS3:Ctrl': [180.0, 260.0],
 'AWA:Bira3Ctrl:Ch03': [-5.0, 5.0],
 'AWA:Bira3Ctrl:Ch04': [-5.0, 5.0],
 'AWA:Bira3Ctrl:Ch05': [-5.0, 5.0],
 'AWA:Bira3Ctrl:Ch06': [-5.0, 5.0],
 'AWALLRF:K1:SetPhase': [339.0, 359.0],
 'AWA:DAC0:Ch08': [4.5, 5.9]}

In [3]:
from xopt import Xopt, Evaluator, VOCS
from xopt.generators.bayesian import BayesianExplorationGenerator, UpperConfidenceBoundGenerator
from xopt.generators.bayesian.models.standard import StandardModelConstructor
from xopt.generators.bayesian.turbo import SafetyTurboController, OptimizeTurboController

import time

def evaluate(inputs):
    env.set_variables(inputs)
    time.sleep(2.0)
    result = env.get_observables(["13ARV1:Sx"])
    result["total_rms_size"] = (result["13ARV1:Sx"]**2 + result["13ARV1:Sy"]**2)**0.5
    return result

# use only one variable
vocs = VOCS(variables=env.variables,
            objectives={"total_rms_size":"MINIMIZE"},
            constraints={
                "13ARV1:bb_penalty":["LESS_THAN",0.0],
                "13ARV1:log10_total_intensity":["GREATER_THAN", 4.5]
            })


### get best point from old data

In [4]:
import yaml
import numpy as np
folder = "analysis/nn_prior/"
f_names = ["exploration_2_nd_filter.yml", "optimization_1.yml", "optimization_2.yml"]
dfs = []
for name in f_names:
    with open(folder + name, "r") as file:
        f = yaml.safe_load(file)
        dfs += [pd.DataFrame(f["data"])]

data = pd.concat(dfs, ignore_index=True)
data = data.dropna(subset=["total_rms_size"])
data["total_rms_size"] = data["total_rms_size"]

old_vocs = VOCS(**f["vocs"])

sorted_idx = np.argsort(data["total_rms_size"].to_numpy())
print(data["total_rms_size"].iloc[sorted_idx])
old_samples = data[vocs.variable_names].iloc[sorted_idx[:30]]


# account for phase drift 103 deg
old_samples["AWALLRF:K1:SetPhase"] = old_samples["AWALLRF:K1:SetPhase"] + 103
old_samples

48      15.790469
96      16.269969
283     18.454183
282     19.338272
284     20.040956
          ...    
174    100.480297
297    104.210789
73     105.367202
111    105.464284
211    110.238814
Name: total_rms_size, Length: 253, dtype: float64


,AWA:Bira3Ctrl:Ch03,AWA:Bira3Ctrl:Ch04,AWA:Bira3Ctrl:Ch05,AWA:Bira3Ctrl:Ch06,AWA:DAC0:Ch08,AWA:Drive:DS1:Ctrl,AWA:Drive:DS3:Ctrl,AWALLRF:K1:SetPhase
48,-0.806105,-0.776121,1.705201,0.131568,4.929338,502.170090,208.105682,353.911686
96,-1.911520,0.936251,0.374213,1.045129,4.887869,533.021678,242.381544,344.579240
283,0.613589,-0.266364,0.856463,-1.384460,5.536879,549.273413,217.433380,345.626308
282,0.590067,-0.293451,0.835026,-1.357757,5.582421,548.718040,214.806440,346.267728
284,0.636957,-0.293168,0.877690,-1.357982,5.490883,549.830475,214.776356,344.977770
329,-0.968191,0.450441,-0.717785,1.360013,5.852420,549.513989,197.374867,346.445403
280,0.566323,-0.321093,0.813333,-1.384709,5.627106,549.268680,212.212888,346.902449
328,-0.974335,0.446432,-0.718588,1.352640,5.858104,549.322819,196.463718,346.367527
322,-0.967682,0.456036,-0.708589,1.362970,5.861005,549.644325,195.530981,346.555834
326,-0.956383,0.448485,-0.714376,1.355010,5.845547,549.617731,195.117060,346.508086


### import nn prior model

### Create Xopt object

In [5]:

model_constructor = StandardModelConstructor(
    use_low_noise_prior=False,
    #mean_modules={"total_rms_size":awa_model}
)
generator = UpperConfidenceBoundGenerator(
    vocs=vocs, 
    model_constructor=model_constructor,
    turbo_controller=OptimizeTurboController(vocs=vocs, length=0.1)
)
generator.numerical_optimizer.n_raw_samples = 512
generator.numerical_optimizer.n_restarts = 512

evaluator = Evaluator(function=evaluate)
X = Xopt(vocs=vocs, evaluator=evaluator, generator=generator)
X.options.dump_file = "run_data_2023_06_19/resampling.yml"

In [6]:
X


            Xopt
________________________________
Version: 1.4.1+50.ge9fc8ac.dirty
Data size: 0
Config as YAML:
xopt: {asynch: false, strict: false, dump_file: run_data_2023_06_19/resampling.yml,
  max_evaluations: null}
generator:
  name: upper_confidence_bound
  model: null
  turbo_controller:
    dim: 8
    batch_size: 1
    length: 0.1
    length_min: 0.0078125
    length_max: 2.0
    failure_counter: 0
    failure_tolerance: 4
    success_counter: 0
    success_tolerance: 4
    center_x: null
    scale_factor: 2.0
    tkwargs: {dtype: torch.float64}
    minimize: true
    best_value: null
  use_cuda: false
  model_constructor:
    name: standard
    use_low_noise_prior: false
    covar_modules: {}
    mean_modules: {}
    trainable_mean_keys: []
    dtype: torch.float64
    device: cpu
  numerical_optimizer: {name: LBFGS, n_raw_samples: 512, n_restarts: 512, max_iter: 2000}
  max_travel_distances: null
  n_monte_carlo_samples: 128
  beta: 2.0
evaluator:
  function: __main__.evalu

### Initialize w/ fixed initial points

In [7]:
# evaluate that point in xopt
X.evaluate_data(old_samples)

CA.Client.Exception...............................................
    Context: "Channel: "13ARV1:image1:ArrayData", Connecting to: 146.139.52.185:5064, Ignored: awa5:5064"
    Source File: ../cac.cpp line 1320
    Current Time: Mon Jun 19 2023 13:56:40.399722644
..................................................................
CA.Client.Exception...............................................
    Context: "Channel: "13ARV1:image1:ArraySize1_RBV", Connecting to: 146.139.52.185:5064, Ignored: awa5:5064"
    Source File: ../cac.cpp line 1320
    Current Time: Mon Jun 19 2023 13:56:40.399913897
..................................................................
CA.Client.Exception...............................................
    Context: "Channel: "13ARV1:image1:ArraySize0_RBV", Connecting to: 146.139.52.185:5064, Ignored: awa5:5064"
    Source File: ../cac.cpp line 1320
    Current Time: Mon Jun 19 2023 13:56:40.400063431
................................................................

,AWA:Bira3Ctrl:Ch03,AWA:Bira3Ctrl:Ch04,AWA:Bira3Ctrl:Ch05,AWA:Bira3Ctrl:Ch06,AWA:DAC0:Ch08,AWA:Drive:DS1:Ctrl,AWA:Drive:DS3:Ctrl,AWALLRF:K1:SetPhase,13ARV1:image1:ArraySize1_RBV,13ARV1:image1:ArraySize0_RBV,...,13ARV1:Cx_std,13ARV1:Cy_std,13ARV1:Sx_std,13ARV1:Sy_std,13ARV1:bb_penalty_std,13ARV1:total_intensity_std,13ARV1:log10_total_intensity_std,total_rms_size,xopt_runtime,xopt_error
1,-0.806105,-0.776121,1.705201,0.131568,4.929338,502.170090,208.105682,353.911686,1200.0,1920.0,...,0.829994,0.229461,0.171975,0.601814,1.213546,417114.485839,0.084282,15.821181,9.251255,False
2,-1.911520,0.936251,0.374213,1.045129,4.887869,533.021678,242.381544,344.579240,1200.0,1920.0,...,0.405735,0.651988,0.250429,0.256740,0.902208,185657.501940,0.040598,15.600330,11.015657,False
3,0.613589,-0.266364,0.856463,-1.384460,5.536879,549.273413,217.433380,345.626308,1200.0,1920.0,...,0.716650,1.454609,0.636275,0.610496,1.572938,479575.896665,0.096419,21.120533,9.699843,False
4,0.590067,-0.293451,0.835026,-1.357757,5.582421,548.718040,214.806440,346.267728,1200.0,1920.0,...,0.560937,0.701598,0.550487,0.266404,0.924189,436647.098985,0.076473,21.366142,12.495802,False
5,0.636957,-0.293168,0.877690,-1.357982,5.490883,549.830475,214.776356,344.977770,1200.0,1920.0,...,0.672733,0.613940,0.243470,0.148704,1.000017,145498.336196,0.028552,23.221382,7.770248,False
6,-0.968191,0.450441,-0.717785,1.360013,5.852420,549.513989,197.374867,346.445403,1200.0,1920.0,...,0.846478,0.340146,0.050326,0.103044,0.550990,272678.665105,0.045972,23.597209,9.234790,False
7,0.566323,-0.321093,0.813333,-1.384709,5.627106,549.268680,212.212888,346.902449,1200.0,1920.0,...,0.369383,1.067088,0.139763,0.125555,1.098699,252015.105055,0.052455,22.089075,7.982702,False
8,-0.974335,0.446432,-0.718588,1.352640,5.858104,549.322819,196.463718,346.367527,1200.0,1920.0,...,0.535709,0.577093,0.128232,0.223792,0.478481,304219.675827,0.050608,24.197822,7.504057,False
9,-0.967682,0.456036,-0.708589,1.362970,5.861005,549.644325,195.530981,346.555834,1200.0,1920.0,...,0.508046,0.959473,0.261136,0.170791,1.076855,463003.159640,0.080247,24.407877,7.034743,False
10,-0.956383,0.448485,-0.714376,1.355010,5.845547,549.617731,195.117060,346.508086,1200.0,1920.0,...,0.609895,0.210581,0.176595,0.137273,0.753773,270718.399506,0.044968,24.482337,7.220196,False


### Run optimization

In [8]:
X.data

,AWA:Bira3Ctrl:Ch03,AWA:Bira3Ctrl:Ch04,AWA:Bira3Ctrl:Ch05,AWA:Bira3Ctrl:Ch06,AWA:DAC0:Ch08,AWA:Drive:DS1:Ctrl,AWA:Drive:DS3:Ctrl,AWALLRF:K1:SetPhase,13ARV1:image1:ArraySize1_RBV,13ARV1:image1:ArraySize0_RBV,...,13ARV1:Cx_std,13ARV1:Cy_std,13ARV1:Sx_std,13ARV1:Sy_std,13ARV1:bb_penalty_std,13ARV1:total_intensity_std,13ARV1:log10_total_intensity_std,total_rms_size,xopt_runtime,xopt_error
1,-0.806105,-0.776121,1.705201,0.131568,4.929338,502.170090,208.105682,353.911686,1200.0,1920.0,...,0.829994,0.229461,0.171975,0.601814,1.213546,417114.485839,0.084282,15.821181,9.251255,False
2,-1.911520,0.936251,0.374213,1.045129,4.887869,533.021678,242.381544,344.579240,1200.0,1920.0,...,0.405735,0.651988,0.250429,0.256740,0.902208,185657.501940,0.040598,15.600330,11.015657,False
3,0.613589,-0.266364,0.856463,-1.384460,5.536879,549.273413,217.433380,345.626308,1200.0,1920.0,...,0.716650,1.454609,0.636275,0.610496,1.572938,479575.896665,0.096419,21.120533,9.699843,False
4,0.590067,-0.293451,0.835026,-1.357757,5.582421,548.718040,214.806440,346.267728,1200.0,1920.0,...,0.560937,0.701598,0.550487,0.266404,0.924189,436647.098985,0.076473,21.366142,12.495802,False
5,0.636957,-0.293168,0.877690,-1.357982,5.490883,549.830475,214.776356,344.977770,1200.0,1920.0,...,0.672733,0.613940,0.243470,0.148704,1.000017,145498.336196,0.028552,23.221382,7.770248,False
6,-0.968191,0.450441,-0.717785,1.360013,5.852420,549.513989,197.374867,346.445403,1200.0,1920.0,...,0.846478,0.340146,0.050326,0.103044,0.550990,272678.665105,0.045972,23.597209,9.234790,False
7,0.566323,-0.321093,0.813333,-1.384709,5.627106,549.268680,212.212888,346.902449,1200.0,1920.0,...,0.369383,1.067088,0.139763,0.125555,1.098699,252015.105055,0.052455,22.089075,7.982702,False
8,-0.974335,0.446432,-0.718588,1.352640,5.858104,549.322819,196.463718,346.367527,1200.0,1920.0,...,0.535709,0.577093,0.128232,0.223792,0.478481,304219.675827,0.050608,24.197822,7.504057,False
9,-0.967682,0.456036,-0.708589,1.362970,5.861005,549.644325,195.530981,346.555834,1200.0,1920.0,...,0.508046,0.959473,0.261136,0.170791,1.076855,463003.159640,0.080247,24.407877,7.034743,False
10,-0.956383,0.448485,-0.714376,1.355010,5.845547,549.617731,195.117060,346.508086,1200.0,1920.0,...,0.609895,0.210581,0.176595,0.137273,0.753773,270718.399506,0.044968,24.482337,7.220196,False
